Variational AutoEncoder by using tf.contrib.distributions.MultivariateNormalDiag
Q:李航 统计学方法 P155 EM算法的M步更新参数公式是怎么推出来的?

In [1]:
import tensorflow as tf
tfd = tf.contrib.distributions
images = tf.placeholder(tf.float32,[None,28,28])

In [ ]:
#设p(z|x)服从标准正态,p(z|x)
def make_prior(code_size = 2):
    mean, stddev = tf.zeros([code_size]),tf.ones([code_size])
    return tfd.MultivariateNormalDiag(mean, stddev)

#猜测q(z)分布的参数
def make_encoder(images, code_size = 2):
    images = tf.layers.flatten(images)
    hidden = tf.layers.dense(images, 100, tf.nn.relu)
    mean = tf.layers.dense(hidden, code_size)
    stddev = tf.layers.dense(hidden, code_size, tf.nn.softplus)
    return tfd.MultivariateNormalDiag(mean, stddev)

def make_decoder(code, data_shape=[28,28]):
    hidden = tf.layers.dense(code, 100, tf.nn.relu)
    logit = tf.layers.dense(hidden, np.prod(data_shape))
    logit = tf.reshape(logit, [-1]+data_shape)
    return tfd.Independent(tfd.Bernoulli(logit),len(data_shape))

In [ ]:
#假设p(z)是标准正态分布
prior = make_prior()
#x->q(z)
posterior = make_encoder(images)
#用sample采样q(z),然后假定p(z|x)=q(z),建模p(x|z)
dist = make_decoder(posterior.sample())

#推断网络目标使q(z)接近p(z|x)等价于maxELBO
#生成网络目标使ELBO接近logP,等价于maxELBO
#ELBO等价于E(期望q)[logP(x|z)]-KL(p(z)||q(z))
#然后什么SGVB AEVB算法求变分下界就orz了
elbo = dist.log_prob(images) - tfd.kl_divergence(posterior, prior)
optimize = tf.train.AdamOptimizer().minimize(-elbo)
#采样生成网络的p(z)得到可视化图片
samples = make_decoder(prior.sample(10)).mean()


既然假设了p(z)服从标准正态,为什么还需要网络的推断部分q(z),就是为了